# Supply Chain Disruption Analysis

## Overview
This report analyzes potential supply chain disruptions for a set of components listed in Figures 8 and 9. The analysis aims to quantify the risk associated with each component, focusing on their criticality to system operations and their role in various configurations.

## Methodology
1. **Data Extraction**: Convert data from Figures into usable formats.
2. **Risk Metrics Calculation**:
   - **Value of Service (VoS)**: Calculates the potential revenue impact of each component's failure.
   - **Time to Recover (TTR)**: Estimates the downtime associated with component failure.
3. **Critical Component Identification**: Highlights components that pose the highest risk based on VoS and TTR.
4. **Disruption Simulation**: Models the operational impact of potential disruptions on supply chain continuity.

## Formulas
- **VoS**: \( VoS_i = \text{Criticality}_i \times \text{Usage Factor}_i \)
- **TTR**: \( TTR_i = \frac{\text{MTTF}_i}{\text{Replacement Rate}_i} \)

Where:
- \( i \) represents each component.
- **Criticality** and **Usage Factor** are derived from operational importance and configuration usage.
- **MTTF** (Mean Time To Failure) and **Replacement Rate** are technical parameters of the components.


## Supply Chain Disruption Risk Management: Analysis and Python Implementation

### Overview

This notebook explores the management of disruption risks in supply chains by analyzing the criticality and reliability of various components within a system. It involves modeling the inventory levels, potential downtimes, and criticality to optimize supply chain resilience and minimize disruptions.

### Problem Statement

We are tasked with assessing the potential impact of disruptions in the supply chain by considering various component vulnerabilities. The objective is to quantify risk in terms of both the criticality to system operations and the potential downtime associated with component failures.

### Data Description

- **Components List**: Detailed information about each component including vendor, cost, criticality, Mean Time to Failure (MTTF), and inventory levels.
- **Configuration Usage**: Specifies how each component is used across different product configurations, impacting the priority and risk management strategy.

### Mathematical Formulations

#### Value of Service (VoS) Calculation
The Value of Service (VoS) for each component is calculated based on its criticality and usage across configurations, reflecting its financial impact on the system:

$$
\text{VoS}_i = \text{Criticality}_i \times \left( \text{Average Inventory Level (Spares)}_i + \text{Average Inventory Level (New Projects)}_i \right)
$$

Where:

- $\text{Criticality}_i$: Importance rating of the component, affecting system functionality.

- $\text{Average Inventory Level (Spares)}_i$ and $\text{Average Inventory Level (New Projects)}_i$: Reflect the stock levels maintained for operational continuity and future projects.

#### Time to Recover (TTR) Estimation
Time to Recover (TTR) is estimated based on the Mean Time to Failure (MTTF) and the replacement rate, indicating the expected downtime:

$$
\text{TTR}_i = \frac{\text{Theoretical MTTF (Hours)}_i}{\text{Replacement Rate/Week}_i}
$$

Where:

- $\text{Theoretical MTTF (Hours)}_i$: The expected operational lifespan of the component before failure.

- $\text{Replacement Rate/Week}_i$: The frequency at which the component is typically replaced, influencing recovery speed.

### Python Implementation

The analysis will be carried out through the following steps:

1. **Data Preparation**: Convert structured data from Figures into pandas DataFrames for manipulation.

2. **Risk Metrics Calculation**: Compute VoS and TTR for each component to identify critical components that could impact the supply chain the most in the event of a disruption.

3. **Critical Component Identification**: Sort components based on VoS and TTR to prioritize risk management efforts.

4. **Visualization**: Display the results graphically to illustrate the risk profile of the supply chain components.

This approach ensures that decision-making is supported by robust quantitative analysis, enabling effective risk mitigation strategies tailored to the specific needs of the supply chain.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data from Figure 8 and Figure 9 combined for analysis
data = {
    "ID": ["ABX001", "ABN002", "GHY456", "KJU879", "PFR934", "JKI984", "JKI990", "JKI768", "BNT162",
           "PRT847", "NDH563", "MBR034", "KTY476", "NMU839", "RTV213", "BLY283", "TTN332", "FBT937",
           "YRE374", "MNA923", "YTH789", "YTH934", "YTH478", "ZVY568", "TYN932", "RVN493", "BBR474"],
    "Name": ["Analog display", "Analog circuit", "Grounding unit", "Circuit braker", "Power filter unit",
             "Connector J984", "Connector J990", "Connector J768", "SwitchBoard", "Power supply unit",
             "Padmount transformer", "Fuse unit", "Fusible switch", "Bus material", "Power distribution panel",
             "Battery unit", "Timer unit", "SB1", "SB2", "SB3", "Adjuster 789", "Adjuster 934", "Adjuster 478",
             "Front panel", "Transmitter", "Receiver", "Back-up receiver"],
    "Criticality": [3, 2, 1, 2, 2, 1, 3, 3, 1, 1, 2, 1, 2, 2, 2, 2, 1, 3, 3, 3, 4, 4, 4, 3, 1, 1, 3],
    "Average Inventory Level (Spares)": [45, 65, 234, 654, 34, 23, 432, 54, 123, 555, 553, 112, 457, 22, 12, 12, 432, 124, 759, 128, 345, 864, 534, 523, 231, 546, 22],
    "Average Inventory Level (New projects)": [88, 214, 243, 532, 125, 311, 122, 111, 765, 252, 235, 353, 324, 642, 124, 435, 1234, 765, 213, 214, 658, 235, 765, 123, 654, 234, 75]
}

# Configuration data from Figure 9
configurations = {
    "ID": ["ABX001", "ABN002", "GHY456", "KJU879", "PFR934", "JKI984", "JKI990", "JKI768", "BNT162",
           "PRT847", "NDH563", "MBR034", "KTY476", "NMU839", "RTV213", "BLY283", "TTN332", "FBT937",
           "YRE374", "MNA923", "YTH789", "YTH934", "YTH478", "ZVY568", "TYN932", "RVN493", "BBR474"],
    "CFG1": [1, 1, 1, 1, 1, 0, 3, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 6, 0, 0, 2, 0, 0, 1, 1, 1, 1],
    "CFG2": [1, 1, 1, 0, 0, 3, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 6, 0, 0, 2, 0, 1, 1, 1, 0],
    "CFG3": [1, 1, 1, 1, 1, 0, 0, 3, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 6, 6, 2, 0, 0, 1, 1, 1, 0],
    "CFG4": [0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 1, 6, 0, 0, 0, 0, 2, 1, 1, 1, 0]
}

# Convert the data to DataFrames for easier manipulation
df_items = pd.DataFrame(data)
df_configs = pd.DataFrame(configurations)

# Merge both DataFrames on ID for a full dataset
df_full = pd.merge(df_items, df_configs, on="ID")

# Calculate Value of Service (VoS) for each item
df_full['VoS'] = df_full['Criticality'] * (df_full['Average Inventory Level (Spares)'] + df_full['Average Inventory Level (New projects)'])

# Calculate theoretical TTR (Time to Recover) if applicable (not all items have a replacement rate)
# This is a simplified example, normally you'd account for logistic and repair times.
df_full['TTR'] = np.where(df_full['Replacement Rate/Week (if known)'].replace('-', np.nan).notna(),
                          df_full['Theoretical MTTF (Hours)'] / df_full['Replacement Rate/Week (if known)'],
                          np.nan)

# Sort items by VoS to see which are most critical
df_sorted_by_vos = df_full.sort_values(by='VoS', ascending=False)

# Display the top 10 critical components
print("Top 10 Critical Components by Value of Service:")
print(df_sorted_by_vos[['ID', 'Name', 'VoS', 'TTR']].head(10))

# Visualization of VoS and TTR for the components
plt.figure(figsize=(14, 7))
plt.scatter(df_full['VoS'], df_full['TTR'], color='blue')
plt.xlabel('Value of Service')
plt.ylabel('Time to Recover (hours)')
plt.title('VoS vs. TTR for Components')
plt.grid(True)
plt.show()

KeyError: 'Replacement Rate/Week (if known)'

## Claude Section

In [1]:
# -*- coding: utf-8 -*-
"""
Supply Chain Disruption Risk Management
---------------------------------------

This code performs calculations for the TTS/TTR methodology to assess supply chain disruption risks for a service provider.

The following data is used:
- Table 1: Number of customers affected by each item (not provided in the code)
- Figure 8: Item data, including average inventory levels and theoretical MTTF
- Figure 9: Item quantities in each configuration
- Figure 10: Forecasted demand for each configuration
- Figure 11: Number of households per market (not used in the code)
- Figure 12: Number of configurations per market (not used in the code)

The code calculates:
1. Value of Service for each item
2. TTS (Time to Survive) for each item
3. Identifies the top 3 items for TTR (Time to Recovery) analysis (manually done based on the results)

"""

import pandas as pd

# Constants
ARPU = 200
WEEKS_PER_YEAR = 52
HOURS_PER_WEEK = 24 * 7

# Load data from Figure 8
fig8_data = {
    'Item ID': ['ABX001', 'ABN002', 'GHY456', 'KJU879', 'PFR934', 'JKI984', 'JKI990', 'JKI768', 'BNT162', 'PRT847', 'NDH563', 'MBR034', 'KTY476', 'NMU839', 'RTV213', 'BLY283', 'TTN332', 'FBT937', 'YRE374', 'MNA923', 'YTH789', 'YTH934', 'YTH478', 'ZVY568', 'TYN932', 'RVN493', 'BBR474'],
    'Theoretical MTTF (Hours)': [5000, 12000, 10000, 5000, 14000, 10000, 17000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 15000, 21300, 123000, 45900, 23000, 12000, 54200, 23400, 12000, 42300, 12000, 43000, 53000],
    'Replacement Rate/Week (if known)': [52, 23, '-', '-', '-', '-', '-', '-', 75, '-', '-', 45, '-', '-', '-', '-', 12, 5, '-', '-', '-', '-', '-', 4, 24, 3, '-'],
    'Average Inventory Level (Spares)': [45, 65, 234, 654, 34, 23, 432, 54, 123, 555, 553, 112, 457, 22, 12, 12, 432, 124, 759, 128, 345, 864, 534, 523, 231, 546, 22],
    'Average Inventory Level (New projects)': [88, 214, 243, 532, 125, 311, 122, 111, 765, 252, 235, 353, 324, 642, 124, 435, 1234, 765, 213, 214, 658, 235, 765, 123, 654, 234, 75]
}

fig8_df = pd.DataFrame(fig8_data)

# Load data from Figure 9
fig9_data = {
    'Item ID': ['ABX001', 'ABN002', 'GHY456', 'KJU879', 'PFR934', 'JKI984', 'JKI990', 'JKI768', 'BNT162', 'PRT847', 'NDH563', 'MBR034', 'KTY476', 'NMU839', 'RTV213', 'BLY283', 'TTN332', 'FBT937', 'YRE374', 'MNA923', 'YTH789', 'YTH934', 'YTH478', 'ZVY568', 'TYN932', 'RVN493', 'BBR474'],
    'CFG1': [1, 1, 1, 1, 1, 0, 3, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 6, 0, 0, 2, 0, 0, 1, 1, 1, 1],
    'CFG2': [1, 1, 1, 0, 0, 3, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 6, 0, 0, 2, 0, 1, 1, 1, 0],
    'CFG3': [1, 1, 1, 1, 1, 0, 0, 3, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 6, 6, 2, 0, 0, 1, 1, 1, 0],
    'CFG4': [0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 1, 6, 0, 0, 0, 0, 2, 1, 1, 1, 0]
}

fig9_df = pd.DataFrame(fig9_data)

# Load data from Figure 10
fig10_data = {
    'Configuration': ['CFG1', 'CFG2', 'CFG3', 'CFG4'],
    'Currently Deployed': [540, 220, 21, 11],
    '1': [82, 23, 54, 4],
    '2': [72, 22, 34, 7],
    '3': [69, 45, 32, 8],
    '4': [75, 48, 22, 8]
}

fig10_df = pd.DataFrame(fig10_data)

# Calculate Value of Service for each item
value_of_service = {
    'Item 23': 4454000 * ARPU,
    'Item 29': 15868000 * ARPU,
    'Item 81': 6489000 * ARPU,
    'Item 101': 32658000 * ARPU
}

print("Value of Service:")
for item, value in value_of_service.items():
    print(f"{item}: ${value:,.0f}")

# Calculate TTS for each item
tts_results = []

for _, row in fig8_df.iterrows():
    item_id = row['Item ID']
    mttf_hours = row['Theoretical MTTF (Hours)']
    replacement_rate = row['Replacement Rate/Week (if known)']
    inventory_spares = row['Average Inventory Level (Spares)']
    inventory_new_projects = row['Average Inventory Level (New projects)']

    if replacement_rate == '-':
        spare_usage_rate = 1 / (mttf_hours / HOURS_PER_WEEK)
    else:
        spare_usage_rate = float(replacement_rate)

    new_projects_usage_rate = 0
    for config, qty in fig9_df[fig9_df['Item ID'] == item_id].iloc[0, 1:].items():
        new_projects_usage_rate += qty * fig10_df[config].sum()

    tts = (inventory_spares + inventory_new_projects) / (spare_usage_rate + new_projects_usage_rate)
    tts_results.append((item_id, tts))

print("\nTime to Survive (TTS):")
for item_id, tts in tts_results:
    print(f"{item_id}: {tts:.2f} weeks")

Value of Service:
Item 23: $890,800,000
Item 29: $3,173,600,000
Item 81: $1,297,800,000
Item 101: $6,531,600,000


KeyError: 'CFG1'